In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install qdrant-client langchain pypdf pypdf2 sentence-transformers tqdm langchain_community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 6.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
!pip install --upgrade langchain

In [2]:
import os
import uuid
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

2025-04-23 19:17:47.616553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745435867.879230      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745435867.957235      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
print("Loading model and connecting to Qdrant...")
model = SentenceTransformer(EMBEDDING_MODEL)

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

🔧 Loading model and connecting to Qdrant...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Create the collection if it doesn’t exist
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE)
)

/tmp/ipykernel_31/2303814936.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [ ]:
for file in tqdm(os.listdir(PDF_FOLDER), desc="Processing PDFs"):
    if not file.endswith(".pdf"):
        continue

    path = os.path.join(PDF_FOLDER, file)
    loader = PyPDFLoader(path)
    pages = loader.load()

    if not pages:
        print(f"Skipping {file}: No content found.")
        continue

    chunks = splitter.split_documents(pages)
    texts = [doc.page_content for doc in chunks]

    if not texts:
        print(f"Skipping {file}: No text after chunking.")
        continue

    embeddings = model.encode(texts).tolist()

    if not embeddings:
        print(f"Skipping {file}: Embedding failed.")
        continue

    points = [
        PointStruct(
            id=str(uuid.uuid4()),
            vector=embeddings[i],
            payload={
                "text": texts[i],
                "source": file
            }
        )
        for i in range(len(texts))
    ]

    if points:
        client.upsert(collection_name=COLLECTION_NAME, points=points)
    else:
        print(f"Skipping {file}: No valid points to upload.")

📂 Processing PDFs:   0%|          | 0/14 [00:00<?, ?it/s]

⚠️  Skipping Reaching_The_Unreached_Brochure_for_ASHA.pdf: No text after chunking.


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

📂 Processing PDFs:  14%|█▍        | 2/14 [00:26<02:40, 13.34s/it]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

📂 Processing PDFs:  86%|████████▌ | 12/14 [02:25<00:24, 12.01s/it]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

📂 Processing PDFs:  93%|█████████▎| 13/14 [02:31<00:10, 10.08s/it]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

📂 Processing PDFs: 100%|██████████| 14/14 [02:54<00:00, 12.49s/it]


In [11]:
query = "clean cloth"
query_vec = model.encode([query])[0]

results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vec,
    limit=5
)

for res in results:
    print(f"{res.payload['source']}: {res.payload['text']}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_31/2877331852.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


ASHA_Induction_Module_English.pdf: Staying clean during menstruation
Commonly women use a cloth which is folded and placed within the 
underwear, or passes over the private parts by means of a string tied around 
the waist. This cloth is washed and reused most of the time. A sanitary napkin 
is a pad worn within the underwear during menstruation to absorb the ﬂ  ow of 
blood. Disposable sanitary napkins are more convenient and easier to use than 
reusable cloth. The blood is absorbed better and there is a feeling of dryness.

book-no-6.pdf: 4) Rinse with clean water
5) Air-dry with hands up and elbow facing the ground (Fig 8)
6) Do not touch with your hands the ground, floor or dirty objects after 
washing your hand
Note:  Use the checklist while observing the skills being implemented. 
When a step is performed correctly, place a tick (/checkbld) in the box.
When a step is not performed correctly, place a cross (X) in the  box.
Make sure to review the steps where crosses appear, so tha